In [188]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

### Load model and data

In [189]:
tf.keras.datasets.mnist.load_data(path="mnist.npz")
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Reshape
x_test = x_test.reshape(10000,784)
org_x_test = x_test/255
# One Hot Encoding
y_test_1hot = np.array(to_categorical(y_test, 10), dtype=np.uint8)
ref_model = tf.keras.models.load_model('saved_model/nn')

### Convert weights, input to maximum uint8 range

In [190]:
def convert_to_max_uint8(v):
    # Normalise
    vmin = np.min(v)
    vmax = np.max(v)
    v = (v-vmin)/(vmax-vmin)
    # Use maximal uint8 range
    v = v * 255
    return np.array(v, dtype=np.float32)
      
# Stack: optional
weights = ref_model.layers[0].weights[0].numpy()
bias = ref_model.layers[0].bias.numpy()
weights = np.vstack((bias, weights))
weights_org = weights

# Convert
conv_weights = convert_to_max_uint8(weights)
conv_x_test = convert_to_max_uint8(x_test)

In [191]:
loss, acc = ref_model.evaluate(org_x_test, y_test_1hot)
print(f"Evaluation auf den Testdaten:\n\nLoss = {loss:.3f}\nAccuracy = {acc:.3f}")

313/313 [==============================] - 2s 6ms/step - loss: 0.2805 - acc: 0.9226
Evaluation auf den Testdaten:

Loss = 0.281
Accuracy = 0.923


### Test conversion error rate

In [192]:
def run_model(x, w):
    x = np.append([255],x)          # 255 for bias cause 1 resembles 255 in uint max scale
    return  np.argmax(np.dot(x,w))  # ! return value is uint64 in size

In [193]:
length = len(x_test)
org_acc_score = 0
conv_acc_score = 0
conv_score = 0
preds = ref_model.predict(org_x_test)
for i in range(length): 
    pred1 = run_model(conv_x_test[i], conv_weights)
    pred2 = np.argmax(preds[i]) 
    if pred1 == np.argmax(y_test_1hot[i]): 
        conv_acc_score += 1
    if pred2 == np.argmax(y_test_1hot[i]): 
        org_acc_score += 1
    if pred1 == pred2:
        conv_score +=1

conv_acc = conv_acc_score/length
org_acc = org_acc_score/length
conv_score = conv_score/length
print(f"Conversion Accuracy: {conv_acc} \nOriginal Accuracy: {org_acc} \nConversion Success: {conv_score}")    

313/313 [==============================] - 2s 5ms/step
Conversion Accuracy: 0.9223 
Original Accuracy: 0.9226 
Conversion Success: 0.9994
